In [ ]:
import numpy as np
import pandas as pd
import geopandas as gp
from datetime import datetime

import shapely

import re
regex = re.compile('^B')
regex_numbers = re.compile(r'\D+')

https://www.baruch.cuny.edu/confluence/pages/viewpage.action?pageId=28016896

In [ ]:
## https://gis.stackexchange.com/questions/309251/how-to-get-equidistant-points-from-a-linestring-geographical-coordinates

def create_geo(busline, points ,time_index):
    '''
    Doc: busline: row indicating busline, should contain geometry
          time_index = pd.date_range obj with required cadance
    '''
    
    busLineFinal = []
    totalIdx = len(time_index)
    lastIdx = 0
    flag = True
    
    
    route_dir = busline[1]
    route_id = busline[0]
    
    #Clip line into 250 unique points
    num_points = points
    points = [busline[6].interpolate(i/float(num_points - 1),
                                                                 normalized=True)
                                                     for i in range(num_points)]    
    #BusLines with reverse routes
    if not route_dir:
        points.reverse()
    busline_geom = gp.GeoSeries(points)
    
    while flag:
        
        busLine = gp.GeoDataFrame(geometry=busline_geom)
        busLine['route_name'] = route_id
        busLine['time'] = np.NaN  
        
        for idx in busLine.index:
            busLine.loc[idx, 'time'] = time_index[lastIdx]
            lastIdx = lastIdx + 1
   
            if lastIdx == totalIdx:
                flag = False
                break
                
        busLineFinal.append(busLine)
    
    return pd.concat(busLineFinal).dropna().reset_index(drop=True)

In [ ]:
gdf = gp.read_file('./bus_routes_nyc_may2019/bus_routes_nyc_may2019.shp')

In [ ]:
brooklynLines = []
points = 250
for row in gdf.values:
    if regex.search(row[0]):
        brooklynLines.append(create_geo(row, points, pd.date_range('00:00', '23:59', freq='30s')))
        break
brooklynPoints = pd.concat(brooklynLines)

In [ ]:
from fiona.crs import from_epsg

brooklynPoints.crs = from_epsg(2263)
brooklynPoints.to_crs(epsg=4326, inplace=True)

In [ ]:
brooklynBuses = brooklynPoints[['time','route_name', 'geometry']].copy()
x, y = brooklynBuses.geometry.x, brooklynBuses.geometry.y

In [ ]:
brooklynBuses.loc[:, 'lon'] = x
brooklynBuses.loc[:, 'lat'] = y
brooklynBuses.drop('geometry', axis=1, inplace=True)
brooklynBuses.head()

POPULATE bus.device table with DUMMY DATA


Insert into the following table using attached shapefile (points along major Brooklyn Bus routes) where device_id = route_id
you will need to create datetime such that each bus route starts at the same time and each point along the time represents an increment of 30 seconds.
Copy this dummy data to cover all "times of day" so that we can load the dummy data into the interface
insert into bus.device (datetime, device_id, lat, lon) VALUES (%s, %s, %s, %s);

In [ ]:
import psycopg2

host = "host"
database = "database"
user = 'user'
password = 'password'

In [ ]:
con = psycopg2.connect(host=host, database=database, user=user, password=password)
cur = con.cursor()

query = 'insert into bus.device (datetime, device_id, lat, lon) VALUES (%s, %s, %s, %s);'
for idx, row in brooklynBuses.iterrows():
    if regex_numbers.findall(row['route_name']):
        cur.execute(query, (datetime.strptime(str(row['time']), '%Y-%m-%d %H:%M:%S'), int(row['route_name'][1:]), row['lat'], row['lon']))

con.close()
